In [2]:
# ===== Step 0: 全局配置 =====
from pathlib import Path

# === 修改成你的本地路径 ===
GENO_PATH = Path("../Data/BYxRM_GenoData.txt")   # 基因型文件
PHENO_PATH = Path("../Data/BYxRM_PhenoData.txt") # 表型文件

# 结果输出目录
OUT_DIR = Path("../Results")  # 也放在内容根下
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 选择性状个数（缺失率最低的前K个）
TOP_K_TRAITS = 12

# 交叉验证配置
N_SPLITS = 5
N_REPEATS = 5
RANDOM_STATE = 42

# PCA 维度（可按需要调整：比如 100/200/300）
PCA_COMPONENTS = 200

print("Config OK.")


Config OK.


In [3]:
# ===== Step 1: 读取与预处理基因型 =====
import pandas as pd
import numpy as np

def load_and_clean_genotype(geno_path: Path,
                            sep: str = "\t",
                            missing_rate_thresh: float = 0.10):
    """
    读取 BY×RM 基因型：
      - 文件格式：TSV，首列 marker，后续列为样本ID，值 ∈ {'R','B'}（可能有少量缺失）
      - 过程：
        1) R/B -> 0/1
        2) 转置为样本×标记
        3) 过滤：位点缺失率 > 阈值剔除；近零方差剔除
        4) 缺失填补：每个位点用众数（最常见等位基因）
    返回：
      X: DataFrame（index=sample_id, columns=markers）
      keep_mask: 被保留的SNP布尔索引
    """
    # 读入（字符）
    df = pd.read_csv(geno_path, sep=sep, dtype=str)
    assert df.columns[0].lower() == "marker", "首列应为 'marker'"
    df = df.set_index(df.columns[0])

    # R/B -> 0/1
    rb_map = {"R": 0, "B": 1}
    df = df.replace(rb_map)

    # 转置为样本×标记
    X = df.T
    X.index.name = "sample_id"

    # 转为数值，无法解析的记为 NaN
    X = X.apply(pd.to_numeric, errors="coerce")

    # 统计位点缺失率与方差
    miss_by_snp = X.isna().mean()
    var_by_snp = X.var(skipna=True)

    # 过滤：缺失率 <= 阈值、方差 > 0
    keep_mask = (miss_by_snp <= missing_rate_thresh) & (var_by_snp > 0.0)
    X = X.loc[:, keep_mask].copy()

    # 众数填补（每列）
    mode_vals = X.mode(dropna=True).iloc[0]
    X = X.fillna(mode_vals)

    # 再保证 dtype 为数值整型/浮点
    X = X.astype(np.float32)

    return X, keep_mask

X, keep_mask = load_and_clean_genotype(GENO_PATH)
print(f"Genotype matrix shape: {X.shape} (samples x kept_SNPs)")


Genotype matrix shape: (1008, 11623) (samples x kept_SNPs)


In [4]:
# ===== Step 2: 读取与处理表型，样本对齐 & 选Top-K性状 =====
def load_and_align_phenotype(pheno_path: Path, X_index):
    # 自动判定分隔符（常见两种）
    try:
        pheno = pd.read_csv(pheno_path, sep="\t")
    except Exception:
        pheno = pd.read_csv(pheno_path, sep=",")

    # 将首列视作样本ID
    if pheno.columns[0].lower() not in ("sample", "sample_id", "strain", "id"):
        # 默认首列为样本ID
        pheno = pheno.set_index(pheno.columns[0])
    else:
        pheno = pheno.set_index(pheno.columns[0])

    # 保留数值型性状
    pheno_num = pheno.select_dtypes(include=[np.number]).copy()

    # 与基因型样本取交集
    common_samples = pheno_num.index.intersection(X_index)
    pheno_num = pheno_num.loc[common_samples]
    return pheno_num

pheno_num = load_and_align_phenotype(PHENO_PATH, X.index)
print(f"Phenotype matrix shape: {pheno_num.shape} (samples x numeric_traits)")

# 计算每个性状缺失率，挑选缺失最少的 TOP_K_TRAITS
missing_rate = pheno_num.isna().mean().sort_values()
selected_traits = missing_rate.head(min(TOP_K_TRAITS, len(missing_rate))).index.tolist()

# 记录并导出
missing_table = pd.DataFrame({
    "missing_rate": missing_rate,
    "n_samples": len(pheno_num),
    "n_non_na": (1 - missing_rate) * len(pheno_num)
}).round({"n_non_na": 0})

missing_table.to_csv(OUT_DIR / "phenotype_missingness.csv")
pd.Series(selected_traits, name="trait").to_csv(OUT_DIR / "selected_traits.csv", index=False)

print("Selected traits:", selected_traits)
print("Saved:", OUT_DIR / "phenotype_missingness.csv", "and", OUT_DIR / "selected_traits.csv")


Phenotype matrix shape: (1008, 46) (samples x numeric_traits)
Selected traits: ['Cobalt_Chloride', 'Tunicamycin', 'YNB', 'Magnesium_Sulfate', 'YPD', 'Maltose', 'Neomycin', 'Menadione', 'YPD:15C', 'x4NQO', 'E6_Berbamine', 'Lactose']
Saved: ../Results/phenotype_missingness.csv and ../Results/selected_traits.csv


In [12]:
# ===== Step 3: 模型与评估：Raw vs PCA =====
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.pipeline import Pipeline

cv = RepeatedKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS, random_state=RANDOM_STATE)

MODELS = {
    "Ridge": Ridge(alpha=1.0, random_state=RANDOM_STATE),
    "LASSO": Lasso(alpha=1e-3, max_iter=10000, random_state=RANDOM_STATE),
    "ENet":  ElasticNet(alpha=1e-3, l1_ratio=0.5, max_iter=10000, random_state=RANDOM_STATE),
    "RF":    RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=RANDOM_STATE),
}

'''def evaluate_trait(X, y, use_pca=False, pca_components=200):
    """
    对单一性状 y 进行模型比较。
    返回：DataFrame (index=['mean_R2','std_R2'], columns=模型名_分支)
    """
    results = {}
    for name, est in MODELS.items():
        steps = []
        # PCA 分支（先降维再喂模型）
        if use_pca:
            steps.append(("pca", PCA(n_components=pca_components, random_state=RANDOM_STATE)))
        # 线性模型建议标准化；树模型可不强制
        if name in ("Ridge", "LASSO", "ENet"):
            steps.append(("scaler", StandardScaler(with_mean=True, with_std=True)))
        steps.append(("model", est))
        pipe = Pipeline(steps)

        scores = cross_val_score(pipe, X, y, scoring="r2", cv=cv, n_jobs=-1)
        results[f"{name}_{'pca' if use_pca else 'raw'}"] = (float(scores.mean()), float(scores.std()))
    return pd.DataFrame(results, index=["mean_R2", "std_R2"])
'''

# === 放在文件前面（或 evaluate_trait 上方）统一声明哪些模型需要标准化 ===
NEED_SCALER = ("Ridge", "LASSO", "ENet", "SVRlin", "SVRrbf")

def evaluate_trait(X, y, use_pca=False, pca_components=200):
    """
    对单一性状 y 进行模型比较。
    返回：DataFrame (index=['mean_R2','std_R2'], columns=模型名_分支)
    """
    results = {}
    for name, est in MODELS.items():
        steps = []
        # 先做 PCA（如果启用），再做标准化（仅对需要的模型）
        if use_pca:
            steps.append(("pca", PCA(n_components=pca_components, random_state=RANDOM_STATE)))
        if name in NEED_SCALER:
            steps.append(("scaler", StandardScaler(with_mean=True, with_std=True)))
        steps.append(("model", est))

        pipe = Pipeline(steps)

        # 有些模型/分支可能在极端数据上报错（例如样本很少或奇异矩阵）
        # 做一次稳健的保护：出错就给 NaN，避免整个流程中断
        try:
            scores = cross_val_score(pipe, X, y, scoring="r2", cv=cv, n_jobs=-1)
            mean_r2, std_r2 = float(scores.mean()), float(scores.std())
        except Exception as e:
            # 你也可以 print(f"[WARN] {name} {'pca' if use_pca else 'raw'} failed: {e}")
            mean_r2, std_r2 = float("nan"), float("nan")

        results[f"{name}_{'pca' if use_pca else 'raw'}"] = (mean_r2, std_r2)

    return pd.DataFrame(results, index=["mean_R2", "std_R2"])


'''追加GBM，SVM和BLUP'''

# ===== Add GBM (LightGBM / XGBoost) & SVM (SVR) =====
from sklearn.svm import SVR

# SVM 两个基线：线性核与 RBF 核（线性核速度更快；RBF 能捕捉非线性）
MODELS["SVRlin"] = SVR(kernel="linear", C=1.0)


# XGBoost（若未安装会跳过）
try:
    from xgboost import XGBRegressor
    MODELS["XGB"] = XGBRegressor(
        n_estimators=600, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        random_state=RANDOM_STATE, n_jobs=-1,
        max_depth=6, reg_lambda=1.0, tree_method="hist"
    )
except Exception as e:
    print("XGBoost 未安装/不可用，已跳过。", e)

print("Models now:", list(MODELS.keys()))


Models now: ['Ridge', 'LASSO', 'ENet', 'RF', 'SVRlin', 'XGB']


In [5]:
# ===== Step 4: 批量评估多个性状，汇总结果（稳健版） =====
def get_clean_xy(X_full: pd.DataFrame, pheno_df: pd.DataFrame, trait: str):
    """
    返回对齐且无NaN的 X_sub, y_sub（仅保留该trait非缺失的样本）。
    """
    # 仅该性状非缺失的样本
    mask = pheno_df[trait].notna()
    # 与 X 的索引再次保险性取交集（通常已对齐，这里双保险）
    valid_idx = pheno_df.index[mask].intersection(X_full.index)

    X_sub = X_full.loc[valid_idx]
    y_sub = pheno_df.loc[valid_idx, trait].astype(float).values
    return X_sub, y_sub, valid_idx

all_rows = []
skipped = []

for trait in selected_traits:
    X_sub, y_sub, idx_used = get_clean_xy(X, pheno_num, trait)

    # 守卫1：样本量必须 >= 折数，否则CV会报错/退化
    if len(y_sub) < N_SPLITS:
        skipped.append((trait, f"n_samples={len(y_sub)} < n_splits={N_SPLITS}"))
        continue

    # 守卫2：y 需要有方差（至少两种不同值）
    if np.unique(y_sub).size < 2:
        skipped.append((trait, "y has zero variance"))
        continue

    # 评估
    res_raw = evaluate_trait(X_sub, y_sub, use_pca=False)
    res_pca = evaluate_trait(X_sub, y_sub, use_pca=True, pca_components=PCA_COMPONENTS)

    out = pd.concat([res_raw, res_pca], axis=1)
    out.insert(0, "trait", trait)
    out.insert(1, "n_samples_used", len(idx_used))
    all_rows.append(out)

# 汇总与保存
if all_rows:
    report = pd.concat(all_rows, axis=0)
    report.to_csv(OUT_DIR / "results_by_trait.csv")
    print("Saved:", OUT_DIR / "results_by_trait.csv")
    display(report.head())
else:
    print("No traits were evaluated. See 'skipped_traits.csv' for reasons.")

# 记录被跳过的性状及原因（如果有）
if skipped:
    skipped_df = pd.DataFrame(skipped, columns=["trait", "reason"])
    skipped_df.to_csv(OUT_DIR / "skipped_traits.csv", index=False)
    print("Some traits were skipped. See:", OUT_DIR / "skipped_traits.csv")


/opt/anaconda3/envs/Final_Project/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.309e-01, tolerance: 6.131e-01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/Final_Project/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.252e-01, tolerance: 6.408e-01
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/Final_Project/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider in

Saved: ../Results/results_by_trait.csv


,trait,n_samples_used,Ridge_raw,LASSO_raw,ENet_raw,RF_raw,SVRlin_raw,XGB_raw,Ridge_pca,LASSO_pca,ENet_pca,RF_pca,SVRlin_pca,XGB_pca
mean_R2,Cobalt_Chloride,1007,0.047073,-0.029337,0.004339,0.392361,0.076903,0.421179,0.420372,0.421283,0.420806,0.166275,0.374690,0.234076
std_R2,Cobalt_Chloride,1007,0.093947,0.111475,0.102711,0.053379,0.091394,0.052117,0.052888,0.052889,0.052896,0.031845,0.058421,0.038479
mean_R2,Tunicamycin,1006,0.354172,0.350660,0.368140,0.381254,0.365300,0.442335,0.578234,0.578613,0.578408,0.179206,0.543120,0.261129
std_R2,Tunicamycin,1006,0.068882,0.077593,0.076429,0.025947,0.069042,0.023950,0.046639,0.046547,0.046600,0.026675,0.048077,0.029942
mean_R2,YNB,1006,0.258673,0.312600,0.288904,0.393094,0.312742,0.439521,0.472253,0.474479,0.473385,0.138064,0.446431,0.197631


In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path

MODEL_NAME_MAP = {
    "Ridge_raw": "Ridge (Raw)", "LASSO_raw": "LASSO (Raw)", "ENet_raw": "Elastic Net (Raw)",
    "RF_raw": "Random Forest (Raw)", "SVRlin_raw": "SVR Linear (Raw)", "XGB_raw": "XGBoost (Raw)",
    "Ridge_pca": "Ridge (PCA)", "LASSO_pca": "LASSO (PCA)", "ENet_pca": "Elastic Net (PCA)",
    "RF_pca": "Random Forest (PCA)", "SVRlin_pca": "SVR Linear (PCA)", "XGB_pca": "XGBoost (PCA)",
}

def plot_r2_bars_from_csv(report_csv, out_png,
                          sort_by="best",   # "best" | "none" | 某个模型列名（如 "Ridge_raw"）
                          show_errorbars=True,
                          annotate_nsamples=True,
                          title="Model comparison across traits",
                          y_min_floor=-0.5):
    df = pd.read_csv(report_csv)

    # 拆分 mean/std 行，并设置索引为 trait
    df_mean = df[df["Unnamed: 0"] == "mean_R2"].set_index("trait").copy()
    df_std  = df[df["Unnamed: 0"] == "std_R2"].set_index("trait").copy() if show_errorbars else None

    # 元信息与模型列
    meta_cols  = ["n_samples_used"]
    model_cols = [c for c in df_mean.columns if c not in meta_cols]

    # ✅ 关键修复：将模型列强制转数值；无法转换的置 NaN，并去掉 inf
    df_mean[model_cols] = (
        df_mean[model_cols]
        .apply(pd.to_numeric, errors="coerce")
        .replace([np.inf, -np.inf], np.nan)
    )
    if df_std is not None:
        df_std[model_cols] = (
            df_std[model_cols]
            .apply(pd.to_numeric, errors="coerce")
            .replace([np.inf, -np.inf], np.nan)
        )

    # 过滤掉全 NaN 的模型列
    model_cols = [c for c in model_cols if not df_mean[c].isna().all()]
    if not model_cols:
        raise ValueError("No valid model columns to plot (all-NaN or missing).")

    # 排序
    if sort_by == "best":
        order = df_mean[model_cols].max(axis=1).sort_values(ascending=False).index
    elif sort_by == "none":
        order = df_mean.index
    else:
        if sort_by not in df_mean.columns:
            raise ValueError(f"sort_by='{sort_by}' not found among columns.")
        order = df_mean[sort_by].sort_values(ascending=False).index

    df_mean = df_mean.loc[order]
    if df_std is not None:
        df_std = df_std.loc[order]

    traits = df_mean.index.values
    x = np.arange(len(traits))
    width = 0.8 / len(model_cols)

    plt.figure(figsize=(max(8, len(traits)*0.7), 6))
    for i, col in enumerate(model_cols):
        heights = df_mean[col].values
        plt.bar(x + i*width, heights, width,
                label=MODEL_NAME_MAP.get(col, col),
                capsize=2 if show_errorbars else 0)

    plt.xticks(x + width*(len(model_cols)-1)/2, traits, rotation=45, ha="right")
    plt.ylabel("R² (CV mean)")
    plt.title(title)

    # 样本量标注（只在第一组柱上）
    if annotate_nsamples and "n_samples_used" in df_mean.columns:
        ns = pd.to_numeric(df_mean["n_samples_used"], errors="coerce").values
        base = df_mean[model_cols[0]].values
        for xi, yi, n in zip(x, base, ns):
            if np.isfinite(yi) and np.isfinite(n):
                plt.text(xi, yi, str(int(n)), ha="center", va="bottom", fontsize=8)

    # 轴限与零线
    plt.axhline(0, linewidth=1)
    y_min = np.nanmin(df_mean[model_cols].values)
    y_min = y_min if np.isfinite(y_min) else 0.0
    plt.ylim(bottom=min(y_min_floor, y_min * 1.1))

    plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.savefig(out_png, dpi=300)
    plt.close()

plot_r2_bars_from_csv(OUT_DIR / "results_by_trait.csv",
                      OUT_DIR / "bar_r2_by_trait.png",
                      sort_by="best", show_errorbars=True, annotate_nsamples=True)


In [29]:
import pandas as pd
from typing import Optional

# 可选：模型优先级（用于并列时决策）
_MODEL_PRIORITY = ["XGB", "RF", "ENet", "Ridge", "LASSO", "SVRlin"]

def best_model_for_trait_robust(report_csv: str, trait_name: str) -> str:
    """
    返回该性状 mean_R2 最高的模型列名（如 'XGB_raw'）。
    - 自动将模型列转为数值（非数值->NaN）
    - 跳过全NaN的模型列
    - 并列时按 _MODEL_PRIORITY 和 Raw>pca 规则打破平局
    """
    df = pd.read_csv(report_csv, index_col=0)
    sub = df[df["trait"] == trait_name]
    if sub.empty:
        raise ValueError(f"Trait '{trait_name}' not found in report CSV.")

    # 只看 mean_R2 行
    if "mean_R2" not in sub.index:
        raise ValueError("Row 'mean_R2' not found for this trait.")
    sub_mean = sub.loc["mean_R2"].copy()

    # 模型列集合
    non_model = [c for c in ("trait", "n_samples_used") if c in sub_mean.index]
    model_cols = [c for c in sub_mean.index if c not in non_model]
    if not model_cols:
        raise ValueError("No model columns found in report.")

    # 转为数值并过滤全NaN
    vals = pd.to_numeric(sub_mean[model_cols], errors="coerce")
    vals = vals.replace([float("inf"), float("-inf")], pd.NA)
    valid = vals.dropna()
    if valid.empty:
        raise ValueError("All model columns are NaN for this trait.")

    # 取最大值集合（可能并列）
    max_val = valid.max()
    ties = valid[valid == max_val].index.tolist()

    # 平局打破：优先 Raw，再按模型优先级
    def _tie_key(col: str):
        name, branch = col.split("_", 1)
        raw_rank = 0 if branch == "raw" else 1
        fam_rank = _MODEL_PRIORITY.index(name) if name in _MODEL_PRIORITY else 999
        return (raw_rank, fam_rank, col)  # 更小更优

    best_col = sorted(ties, key=_tie_key)[0]
    return best_col


In [30]:
from sklearn.model_selection import RepeatedKFold
from sklearn.base import clone
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# 与主流程一致
NEED_SCALER = ("Ridge", "LASSO", "ENet", "SVRlin", "SVRrbf")

def plot_pred_vs_true_subset(X_full, pheno_df, trait_name, model_key, out_png,
                             n_splits=5, n_repeats=1, random_state=42, pca_components=200,
                             save_oof_csv: Optional[str] = None):
    """
    在该性状有效样本子集上，按指定 model_key（如 'XGB_raw'）生成 OOF 预测散点图。
    同时可选导出 OOF 预测 CSV（idx, y_true, y_pred, fold）。
    """
    # 1) 与 Step 4 一致的子集
    X_sub, y_sub, idx_used = get_clean_xy(X_full, pheno_df, trait_name)
    if len(np.unique(y_sub)) < 2:
        raise ValueError("Target has zero variance in the selected subset.")

    # 2) 构建与 evaluate_trait 相同逻辑的 pipeline
    try:
        name, branch = model_key.split("_", 1)
    except ValueError:
        raise ValueError(f"model_key '{model_key}' must look like 'Model_branch' (e.g., 'XGB_raw').")

    if name not in MODELS:
        raise KeyError(f"Model '{name}' not found in MODELS. Available: {list(MODELS.keys())}")

    est = clone(MODELS[name])
    steps = []
    if branch == "pca":
        steps.append(("pca", PCA(n_components=pca_components, random_state=random_state)))
    if name in NEED_SCALER:
        steps.append(("scaler", StandardScaler(with_mean=True, with_std=True)))
    steps.append(("model", est))
    pipe = Pipeline(steps)

    # 3) OOF 预测
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
    y_true_all, y_pred_all, fold_all, idx_all = [], [], [], []

    X_arr = X_sub.to_numpy()
    for fold_id, (tr_idx, te_idx) in enumerate(rkf.split(X_arr), start=1):
        pipe.fit(X_arr[tr_idx], y_sub[tr_idx])
        y_pred = pipe.predict(X_arr[te_idx])
        y_true_all.append(y_sub[te_idx])
        y_pred_all.append(y_pred)
        fold_all.extend([fold_id] * len(te_idx))
        idx_all.extend(idx_used.to_numpy()[te_idx])

    y_true_all = np.concatenate(y_true_all)
    y_pred_all = np.concatenate(y_pred_all)

    r2 = r2_score(y_true_all, y_pred_all)
    rmse = mean_squared_error(y_true_all, y_pred_all, squared=False)

    # 4) 作图
    plt.figure(figsize=(6, 6))
    plt.scatter(y_true_all, y_pred_all, alpha=0.5, s=12)
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.title(f"{trait_name} | {model_key} | OOF R²={r2:.3f}, RMSE={rmse:.3f}")
    lims = [min(y_true_all.min(), y_pred_all.min()), max(y_true_all.max(), y_pred_all.max())]
    plt.plot(lims, lims, linewidth=1)
    plt.tight_layout()
    out_png = Path(out_png)
    plt.savefig(out_png, dpi=300)
    plt.close()
    print("Saved:", out_png)

    # 5) 可选：导出 OOF 预测
    if save_oof_csv:
        oof_df = pd.DataFrame({
            "sample_id": idx_all,
            "fold": fold_all,
            "y_true": y_true_all,
            "y_pred": y_pred_all,
        })
        oof_df.to_csv(save_oof_csv, index=False)
        print("Saved OOF:", save_oof_csv)

    return r2, rmse


trait0 = selected_traits[0]
best0 = best_model_for_trait_robust(OUT_DIR / "results_by_trait.csv", trait0)

r2, rmse = plot_pred_vs_true_subset(
    X, pheno_num, trait0, best0,
    OUT_DIR / f"scatter_{trait0}_{best0}.png",
    n_splits=N_SPLITS, n_repeats=1, random_state=RANDOM_STATE, pca_components=PCA_COMPONENTS,
    save_oof_csv=OUT_DIR / f"oof_{trait0}_{best0}.csv"
)
print(trait0, best0, r2, rmse)


Saved: ../Results/scatter_Cobalt_Chloride_LASSO_pca.png
Saved OOF: ../Results/oof_Cobalt_Chloride_LASSO_pca.csv
Cobalt_Chloride LASSO_pca 0.427132883496378 2.124883728816087


In [35]:
# === 生成“最佳模型小表 / 模型汇总 / Δ(PCA−Raw) 图表” ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

base = Path(OUT_DIR)  # 或 Path("/mnt/data")
report_csv = base / "results_by_trait.csv"

df = pd.read_csv(report_csv)
df_mean = df[df["Unnamed: 0"] == "mean_R2"].set_index("trait").copy()
df_std  = df[df["Unnamed: 0"] == "std_R2"].set_index("trait").copy()

meta_cols = ["n_samples_used"]
model_cols = [c for c in df_mean.columns if c not in meta_cols]

# 数值化
df_mean[model_cols] = df_mean[model_cols].apply(pd.to_numeric, errors="coerce")
df_std[model_cols]  = df_std[model_cols].apply(pd.to_numeric, errors="coerce")

# 1) 每个性状的最佳模型（含次优与差值）
best_model = df_mean[model_cols].idxmax(axis=1)              # 每行对应的最佳“列名”
# ---- 用 numpy 索引替代 .lookup ----
row_idx = np.arange(len(df_mean))
col_idx_best = df_mean.columns.get_indexer(best_model)       # 把“列名”映射成列下标
best_score = df_mean.to_numpy()[row_idx, col_idx_best]       # 取出最佳分数
best_std   = df_std.to_numpy()[row_idx, col_idx_best]        # 取出最佳的 std

# 计算次优与差值
second_best_model, second_best_score = [], []
for tr, row in df_mean[model_cols].iterrows():
    vals = row.dropna().sort_values(ascending=False)
    if len(vals) >= 2:
        second_best_model.append(vals.index[1])
        second_best_score.append(vals.iloc[1])
    else:
        second_best_model.append(np.nan)
        second_best_score.append(np.nan)

best_tbl = pd.DataFrame({
    "trait": df_mean.index,
    "n_samples_used": df_mean["n_samples_used"].astype(int).values,
    "best_model": best_model.values,
    "best_mean_R2": best_score,
    "best_std_R2": best_std,
    "second_best_model": second_best_model,
    "second_best_mean_R2": second_best_score,
    "delta_best_minus_second": np.array(best_score) - np.array(second_best_score)
}).sort_values("best_mean_R2", ascending=False)
best_tbl.to_csv(base / "best_model_by_trait.csv", index=False)

# 2) 跨性状模型汇总
# ==== 先在 df_mean/df_std 上清理无效模型列 ====
# （放在计算 summary 之前）
df_mean[model_cols] = df_mean[model_cols].replace([np.inf, -np.inf], np.nan)
df_std[model_cols]  = df_std[model_cols].replace([np.inf, -np.inf], np.nan)

valid_cols = [c for c in model_cols if not df_mean[c].isna().all()]
df_mean = df_mean[valid_cols]
df_std  = df_std[valid_cols]
model_cols = valid_cols

# ==== 2) 跨性状模型汇总（均值/中位/标准差/排名） ====
means   = df_mean[model_cols].mean(axis=0, skipna=True)
medians = df_mean[model_cols].median(axis=0, skipna=True)
stds    = df_mean[model_cols].std(axis=0, skipna=True)

summary = pd.DataFrame({
    "mean_R2": means,
    "median_R2": medians,
    "std_R2_across_traits": stds,
    "n_traits": df_mean.shape[0],
})

# 用可空整型避免 NaN 转 int 报错；或改用 fillna 后转 int
ranks = (-summary["mean_R2"]).rank(method="dense", na_option="bottom")
summary["rank_by_mean"] = ranks.astype("Int64")  # 保留缺失为 <NA>
# 若你更想要纯 int：取消上面一行，改用下面两行
# ranks = ranks.where(ranks.notna(), ranks.max() + 1)
# summary["rank_by_mean"] = ranks.astype(int)

summary = summary.sort_values("mean_R2", ascending=False)
summary.to_csv(base / "model_summary_across_traits.csv")


# 3) Δ(PCA − Raw) 表 + 图
families = ["Ridge", "LASSO", "ENet", "RF", "SVRlin", "XGB"]
delta_data = {}
for fam in families:
    raw, pca = f"{fam}_raw", f"{fam}_pca"
    if raw in df_mean.columns and pca in df_mean.columns:
        delta_data[f"{fam}_delta"] = df_mean[pca] - df_mean[raw]
delta_df = pd.DataFrame(delta_data, index=df_mean.index)
delta_df.to_csv(base / "delta_pca_minus_raw.csv")

plt.figure(figsize=(8,5))
plt.boxplot([delta_df[c].dropna().values for c in delta_df.columns], showmeans=True)
plt.axhline(0, linewidth=1)
plt.xticks(np.arange(1, len(delta_df.columns)+1), [c.replace("_delta","") for c in delta_df.columns])
plt.ylabel("Δ R² (PCA − Raw)")
plt.title("Effect of PCA across traits")
plt.tight_layout()
plt.savefig(base / "delta_pca_minus_raw_boxplot.png", dpi=300)
plt.close()

# Raw vs PCA 配对散点
for fam in families:
    raw, pca = f"{fam}_raw", f"{fam}_pca"
    if raw in df_mean.columns and pca in df_mean.columns:
        x, y = df_mean[raw].values, df_mean[pca].values
        mask = np.isfinite(x) & np.isfinite(y)
        r = np.corrcoef(x[mask], y[mask])[0,1] if mask.sum() > 1 else np.nan
        plt.figure(figsize=(5,5))
        plt.scatter(x, y, alpha=0.6, s=16)
        lims = [min(np.nanmin(x), np.nanmin(y)), max(np.nanmax(x), np.nanmax(y))]
        plt.plot(lims, lims, linewidth=1)
        plt.xlabel("Raw R²"); plt.ylabel("PCA R²")
        plt.title(f"{fam}: Raw vs PCA (r={r:.2f})")
        plt.tight_layout()
        plt.savefig(base / f"scatter_raw_vs_pca_{fam}.png", dpi=300)
        plt.close()
